In [1]:
import pandas as pd

df = pd.read_csv(r"C:\IMU-GAIT-FALL-DETECTION\IMU-Gait-Fall-Detection\imu-gait-fall-detection\data\processed\WISDM_combined.csv")
df.head()

,ACTIVITY,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,ZMFCC11,ZMFCC12,XYCOS,XZCOS,YZCOS,XYCOR,XZCOR,YZCOR,RESULTANT,class
0,A,0.235,0.470,0.275,0.020,0.000,0.0,0.0,0.0,0.0,...,0.486106,0.479859,-0.550668,0.049864,0.121354,-0.251024,0.164468,-0.110722,10.05180,1600
1,A,0.275,0.440,0.270,0.015,0.000,0.0,0.0,0.0,0.0,...,0.479571,0.473409,-0.633171,0.072129,0.161492,-0.386416,0.215680,-0.034375,10.11710,1600
2,A,0.320,0.430,0.245,0.000,0.005,0.0,0.0,0.0,0.0,...,0.483005,0.476798,-0.659493,0.087043,0.162157,-0.325151,0.272380,-0.077274,9.98384,1600
3,A,0.315,0.495,0.185,0.005,0.000,0.0,0.0,0.0,0.0,...,0.480711,0.474534,-0.712081,0.003810,0.210015,-0.364285,0.203131,0.015328,10.10600,1600
4,A,0.215,0.455,0.325,0.005,0.000,0.0,0.0,0.0,0.0,...,0.468836,0.462811,-0.534933,0.047553,0.275833,-0.216423,0.238500,-0.009870,10.05210,1600


In [2]:
X = df.drop(['ACTIVITY'], axis=1)
y = df['ACTIVITY']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}

rf = RandomForestClassifier(random_state=42)

grid = GridSearchCV(rf, params, cv=3, n_jobs=-1, verbose=2)
grid.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [10, 20, None],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             verbose=2)

In [5]:
best_rf = grid.best_estimator_
best_rf

RandomForestClassifier(n_estimators=300, random_state=42)

In [6]:
from sklearn.metrics import accuracy_score

rf_tuned_test_acc = accuracy_score(y_test, best_rf.predict(X_test))

print("Tuned RF Test Accuracy:", rf_tuned_test_acc)

Tuned RF Test Accuracy: 0.7632490013315579


In [7]:
import pandas as pd

results = {
    'Model': [
        'Random Forest (Baseline)',
        'XGBoost (Baseline)',
        'Random Forest (PCA)',
        'Random Forest (Tuned)'
    ],
    'Accuracy': [
        0.7612,
        0.7906,
        0.7574,
        rf_tuned_test_acc
    ]
}

results_df = pd.DataFrame(results)
results_df

,Model,Accuracy
0,Random Forest (Baseline),0.761200
1,XGBoost (Baseline),0.790600
2,Random Forest (PCA),0.757400
3,Random Forest (Tuned),0.763249
